# Digit Recognizer - MNIST
##### https://www.kaggle.com/competitions/digit-recognizer

### Imports


In [ ]:
import pandas as pd
import numpy as np

### Load Data

In [ ]:
# In each row, first column contains a label (digit). The remainder 784 columns (28px x 28px image) contain pixel values of 0-255.
mnist_df = pd.read_csv("data/train.csv")
mnist_df.head()

In [ ]:
mnist_df = mnist_df.sample(frac=1, axis=0)
mnist_df

### Transform Data

In [ ]:
# Transpose the data so that the first row will be equal to all the labels and the remainder of rows in each column will be the corresponding pixel value
mnist_df_T = mnist_df.T
mnist_df_T.head()

In [ ]:
# Create NumPy array from Pandas dataframe
mnist_arr = mnist_df_T.values

In [ ]:
mnist_arr

In [ ]:
# Shuffle columns
# We could also do np.random.shufflle(mnist_arr.T) but below is supposed to be faster
#mnist_arr = mnist_arr[:, np.random.permutation(mnist_arr.shape[1])]
#mnist_arr

In [ ]:
# Transform a label so that 3 is represented by [0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0] instead of just 3
def transform_label(label):
    desired_output = np.zeros((10, 1))
    desired_output[label] = 1.0
    return desired_output

In [ ]:
labels = mnist_arr[0, :]
labels

In [ ]:
# Transform our labels so that 3 is represented by [0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0] instead of just 3
# Each column is one example
desired_outputs = np.array([transform_label(l) for l in labels]).T.reshape((10, 42000))
desired_outputs

In [ ]:
# Get all rows except the first one (because it contains labels), each column is one example
# Since the pixelvalues are 0-255 we divide them by 255 to get values between 0 and 1 as activations of the input layer
# Each row is one example
pixel_rows = (mnist_arr[1:, :] / 255.0).T
pixel_rows

### Setup of weights and biases
##### This will depend on the # of layers and the # of neurons in each of them.
##### We will have 1 hidden layer with 30 neurons and 1 output layer with 10 neurons
##### i.e. Network([784, 30, 10])

In [ ]:
class Network:
    def __init__(self, layers):
        self.no_of_layers = len(layers)
        self.layer_sizes = layers
        self.weights = self.__init_weights()
        self.biases = self.__init_biases()

    def __init_weights(self):
        # (784,30) and (30, 10) will be zipped, we will create two matrices with dimensions 30x784 and 10x30
        return [np.random.randn(y, x) for x, y in zip(self.layer_sizes[:-1], self.layer_sizes[1:])]

    def __init_biases(self):
        # two arrays with 30, and 10 random numbers each
        return [np.random.randn(x) for x in self.layer_sizes[1:]]

    def __calculate_weighted_sums(self, activations, layer):
        # weights is (30, 784) and activations is (784, X), the result will be (30, X)
        # biases is (30, ), we need to transpose result so that it is (42000, 30), this way we can add biases
        return (np.dot(self.weights[layer-1], activations).T + self.biases[layer-1]).T

    # For activation we will use the sigmoid function
    def calculate_activations(self, activations, layer):
        z = self.__calculate_weighted_sums(activations, layer)
        return self.sigmoid(z)

    def feedforward(self, input_layer):
        # Calculate activations for each layer.
        # A contain activations in respective layers for each example
        activations = [input_layer.T]
        for l in range(self.no_of_layers-1):
            activations.append(network.calculate_activations(activations[l], l+1))
        return activations[1:]

    def calculate_errors(self, activations, expected):
        errors = [network.cost_derivative(activations[-1], expected) * network.sigmoid_derivative(activations[-1])]
        for l in range(0, len(activations)-1)[::-1]:
            errors.append(np.dot(self.weights[l+1].T, errors[-1]) * network.sigmoid_derivative(activations[l]))
        return errors[::-1]

    @staticmethod
    def calculate_weight_deltas(activations, errors, mini_batch_size):
        return [np.dot(e, a.T) / mini_batch_size for e, a in zip(errors, activations[0:len(activations)-1])]

    @staticmethod
    def calculate_bias_deltas(errors):
            return [np.mean(e, 1) for e in errors]

    def adjust_weights(self, weight_deltas, alpha):
        self.weights = [self.weights[i] + (weight_deltas[i] * alpha) for i in range(len(weight_deltas))]

    def adjust_biases(self, bias_deltas, alpha):
        self.biases = [self.biases[i] + (bias_deltas[i] * alpha) for i in range(len(bias_deltas))]

    @staticmethod
    def sigmoid(z):
        # Sigmoid function, applied elementwise if z is a matrix
        return 1 / (1 + np.exp(z))

    @staticmethod
    def sigmoid_derivative(activations):
        # sigmoid(z) * (1 - sigmoid(z))
        # sigmoig(z) = activation so we can do it like below
        return activations * (1 - activations)

    @staticmethod
    def cost(output, expected):
        return np.power((output-expected)/2, 2)

    @staticmethod
    def cost_derivative(output, expected):
        return output - expected

In [ ]:
network = Network([784, 30, 10])

## Stochastic Gradient Descent

In [ ]:
def calculate_accuracy(activations, actual):
    max_indices = np.argmax(activations[-1], axis=0)
    x = actual - max_indices
    correct = x == 0
    print(x[correct].size / actual.size)

In [ ]:
def SGD(mini_batch_size, epochs, alpha):
    for epoch in range(epochs):
        random_number = np.random.randint(mini_batch_size, 42001)
        inputs = pixel_rows[random_number - mini_batch_size : random_number, :]
        activations = network.feedforward(inputs)
        if epoch % 100 == 0:
            calculate_accuracy(activations, labels[random_number - mini_batch_size : random_number])
        errors_in_layers = network.calculate_errors(activations, desired_outputs[:, random_number - mini_batch_size : random_number])
        inputs_and_activations = [inputs.T] + activations
        weight_deltas = network.calculate_weight_deltas(inputs_and_activations, errors_in_layers, mini_batch_size)
        bias_deltas = network.calculate_bias_deltas(errors_in_layers)
        network.adjust_weights(weight_deltas, alpha)
        network.adjust_biases(bias_deltas, alpha)

In [ ]:
SGD(1000, 10000, 0.1)

In [ ]:
# # Calculate activations in the first and second layer.
# # A1 & A2 contain activations in respective layers for each example
# A1 = network.calculate_activations(pixel_rows.T, 1)
# A2 = network.calculate_activations(A1, 2)

In [ ]:
# error_in_A2_layer = network.cost_derivative(A2, desired_outputs) * network.sigmoid_derivative(A2)
# error_in_A1_layer = np.dot(network.weights[1].T, error_in_A2_layer) * network.sigmoid_derivative(A1)

In [ ]:
# # Gradient descent
# deltaW2 = np.dot(error_in_A2_layer, A1.T) / 42000
# deltaW2

In [ ]:
# deltaW1 = np.dot(error_in_A1_layer, pixel_rows) / 42000
# deltaW1

In [ ]:
# deltaB2 = np.mean(error_in_A2_layer, 1)
# deltaB2

In [ ]:
# deltaB1 = np.mean(error_in_A1_layer, 1)
# deltaB1